In [ ]:
Імпортуємо необхідні бібліотеки

In [28]:
import os
import cv2
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from pprintpp import pprint as pp
import random

In [11]:
import warnings
# Вимкнути всі попередження для видаленн зайвої інформації з консолі
warnings.filterwarnings('ignore')

In [12]:
def compute_hog_features(image_paths, hog_descriptor):
    features = []
    for img_path in image_paths:
        # Read the image in grayscale
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        if img is None:
            print(f"Warning: Unable to read image {img_path}. Skipping.")
            continue
        # Resize image to ensure consistent feature size (optional, depending on your use case)
        img = cv2.resize(img, (64, 128))
        
        # Compute HOG features
        hog_features = hog_descriptor.compute(img)
        
        # Flatten features and add to the list
        features.append(hog_features.flatten())
    
    return np.array(features)

In [13]:
def create_labels_from_folder(folder_path):
    """
    Генерує мітки для зображень на основі назви папки.
    
    Parameters:
        folder_path (str): Шлях до папки, що містить зображення.
        
    Returns:
        image_paths (list): Список шляхів до зображень.
        labels (list): Список міток (назва папки для кожного зображення).
    """
    image_paths_labels = []
    labels = []
    folder_name = os.path.basename(os.path.normpath(folder_path))  # Назва папки як мітка
    for file_name in os.listdir(folder_path):
        full_path = os.path.join(folder_path, file_name)
        if os.path.isfile(full_path):
            image_paths_labels.append(full_path)
            labels.append(folder_name)  # Додаємо назву папки як мітку
    return image_paths_labels, labels

In [41]:
# Перевірка результату
check_data_1=create_labels_from_folder("Images/RSSCN7/fResident")

img_paths_1=check_data_1[0]
labels_1=check_data_1[1]
img_paths_con=img_paths_1
labels_con=labels_1

check_data_2=create_labels_from_folder("Images/RSSCN7/gParking")

img_paths_con_2=check_data_2[0]
labels_con_2=check_data_2[1]
img_paths_con.extend(img_paths_con_2)
labels_con.extend(labels_con_2)

#pp(img_paths_con)

sub_imgs_false_dataset = list(zip(img_paths_con, labels_con))
random.shuffle(sub_imgs_false_dataset)
sub_imgs_false_dataset_imgs = [item[0] for item in sub_imgs_false_dataset]
sub_imgs_false_dataset_labels = [item[1] for item in sub_imgs_false_dataset]
pp(sub_imgs_false_dataset_labels)
sub_imgs_false_dataset_labels_bin = [1 if label == 'gParking' else 0 for label in sub_imgs_false_dataset_labels]

pp(sub_imgs_false_dataset_labels_bin)
#pp(sub_imgs_false_dataset_imgs)
#pp(sub_imgs_false_dataset)

#pp(sub_imgs_false_dataset[0])

[
    'gParking',
    'fResident',
    'fResident',
    'fResident',
    'gParking',
    'gParking',
    'fResident',
    'fResident',
    'gParking',
    'fResident',
    'fResident',
    'fResident',
    'gParking',
    'fResident',
    'gParking',
    'fResident',
    'gParking',
    'fResident',
    'gParking',
    'gParking',
    'gParking',
    'gParking',
    'fResident',
    'fResident',
    'fResident',
    'gParking',
    'gParking',
    'fResident',
    'gParking',
    'gParking',
    'fResident',
    'fResident',
    'gParking',
    'fResident',
    'fResident',
    'gParking',
    'gParking',
    'gParking',
    'fResident',
    'fResident',
    'fResident',
    'fResident',
    'fResident',
    'fResident',
    'gParking',
    'fResident',
    'fResident',
    'fResident',
    'fResident',
    'fResident',
    'fResident',
    'gParking',
    'gParking',
    'gParking',
    'gParking',
    'fResident',
    'gParking',
    'fResident',
    'fResident',
    'gParking',
    

In [25]:
def load_images_from_folder(folder):
    image_paths = []
    for file_name in os.listdir(folder):
        full_path = os.path.join(folder, file_name)
        if os.path.isfile(full_path):
            image_paths.append(full_path)
    return image_paths

In [26]:
# Перевірка
image_folder="Images/RSSCN7/fResident"
image_paths=load_images_from_folder(image_folder)
print(image_paths)

['Images/RSSCN7/fResident/f177.jpg', 'Images/RSSCN7/fResident/f163.jpg', 'Images/RSSCN7/fResident/f188.jpg', 'Images/RSSCN7/fResident/f375.jpg', 'Images/RSSCN7/fResident/f361.jpg', 'Images/RSSCN7/fResident/f349.jpg', 'Images/RSSCN7/fResident/f201.jpg', 'Images/RSSCN7/fResident/f215.jpg', 'Images/RSSCN7/fResident/f229.jpg', 'Images/RSSCN7/fResident/f003.jpg', 'Images/RSSCN7/fResident/f017.jpg', 'Images/RSSCN7/fResident/f016.jpg', 'Images/RSSCN7/fResident/f002.jpg', 'Images/RSSCN7/fResident/f228.jpg', 'Images/RSSCN7/fResident/f214.jpg', 'Images/RSSCN7/fResident/f200.jpg', 'Images/RSSCN7/fResident/f348.jpg', 'Images/RSSCN7/fResident/f360.jpg', 'Images/RSSCN7/fResident/f374.jpg', 'Images/RSSCN7/fResident/f189.jpg', 'Images/RSSCN7/fResident/f162.jpg', 'Images/RSSCN7/fResident/f176.jpg', 'Images/RSSCN7/fResident/f160.jpg', 'Images/RSSCN7/fResident/f174.jpg', 'Images/RSSCN7/fResident/f148.jpg', 'Images/RSSCN7/fResident/f362.jpg', 'Images/RSSCN7/fResident/f376.jpg', 'Images/RSSCN7/fResident/f3

In [34]:
def classify_with_hog_and_xgboost(image_paths, labels):
    # Load image paths
    #image_paths = load_images_from_folder(image_folder)

    if len(image_paths) != len(labels):
        raise ValueError("Number of labels does not match number of images.")

    # Initialize HOG descriptor
    win_size = (64, 128)  # Size of the image window
    block_size = (16, 16)  # Size of the blocks
    block_stride = (8, 8)  # Stride for block movement
    cell_size = (8, 8)  # Size of the cells
    nbins = 9  # Number of histogram bins

    hog_descriptor = cv2.HOGDescriptor(
        win_size,
        block_size,
        block_stride,
        cell_size,
        nbins
    )

    # Compute HOG features for all images
    print("Computing HOG features...")
    features = compute_hog_features(image_paths, hog_descriptor)
    
    # Convert labels to numpy array
    labels = np.array(labels)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

    # Train XGBoost classifier
    print("Training XGBoost classifier...")
    model = xgb.XGBClassifier(
        objective='binary:logistic',  # Use appropriate objective for your task (e.g., 'multi:softmax' for multi-class classification)
        n_estimators=100,
        max_depth=6,
        eta=0.3
    )
    model.fit(X_train, y_train)

    # Make predictions
    print("Making predictions...")
    y_pred = model.predict(X_test)

    # Compute accuracy
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy * 100:.2f}%")

    return model

In [42]:
# Example usage:
if __name__ == "__main__":
    # Folder containing images
    # image_folder = "path/to/image/folder"

    # # Labels corresponding to the images (example binary labels, adjust as needed)
    # labels = [0, 1, 1, 0, 1]  # Replace with your actual labels

    # # Ensure labels match the number of images in the folder before running
    # if len(labels) != len(os.listdir(image_folder)):
    #     raise ValueError("Number of labels must match the number of images in the folder.")
    
    # Train and evaluate classifier
    model = classify_with_hog_and_xgboost(sub_imgs_false_dataset_imgs, sub_imgs_false_dataset_labels_bin)

Computing HOG features...
Training XGBoost classifier...
Making predictions...
Accuracy: 78.75%


## Складові коду:

    Функція compute_hog_features: розраховує HOG дескриптори для всіх зображень, які зчитуються із зазначеної папки. Для кожного зображення дескриптор розраховується і зберігається у вигляді вектора ознак.
    Модель XGBoost: використовується для навчання на базі цих ознак.
    Простий тест/тренувальний процес: після розбиття даних на тренувальну та тестову частини, здійснюється навчання і оцінювання моделі.

Підготування:

    Зображення мають бути однакового розміру або мають бути попередньо приведені до стандартного розміру (в цьому коді розмір встановлено як 64x128 пікселів).
    Мітки мають відповідати кількості зображень у папці.

Якщо класифікація є багатокласовою, вам потрібно налаштувати параметр objective в XGBClassifier на 'multi:softmax' і передати кількість класів параметром num_class.


# Задача

